# 题目

> 给定两个整数数组 `preorder` 和 `inorder` ，其中 `preorder` 是二叉树的前序遍历， `inorder` 是同一棵树的中序遍历，请构造二叉树并返回其根节点。

# 方法一：递归

> 二叉树的前序遍历为：根节点-左子节点-右子节点；二叉树的中序遍历为：左子节点-根节点-右子节点。  
通过**前序遍历**可以快速**确定二叉树的根节点（列表中的首个元素）**。  
将中序遍历的列表制作成哈希表，列表元素（节点的值）为哈希表的key，其对应的列表位置为哈希表的value，以方便查找。  
确定树/子树的根节点后，可以使用**中序遍历**列表确定**左右子树的节点个数（根节点左边的元素为左子树节点，右边元素为右子树节点）**。  
从树的根节点开始，按照根节点-左子树根节点-右子树根节点的顺序以递归的形式返回根节点。

## 复杂度

- 时间复杂度: $O(n)$ ，其中 $n$ 是树中的节点个数。

> 创建了 $n$ 个节点。

- 空间复杂度: $O(n)$ ，其中 $n$ 是树中的节点个数。

> 除去返回的答案需要的 $O(n)$ 空间之外，我们还需要使用 $O(n)$ 的空间存储哈希映射，以及 $O(h)$（其中 $h$ 是树的高度）的空间表示递归时栈空间。这里 $h<n$ ，所以总空间复杂度为 $O(n)$ 。

## 代码

In [1]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [2]:
def buildTree(preorder, inorder):
    def myBuildTree(preorder_left, preorder_right, inorder_left, inorder_right):  # 输入两个列表最左边和最右边元素的索引
        if preorder_left > preorder_right:
            return None
        
        # 前序遍历中的第一个节点就是根节点，给出根节点在前序遍历中的位置索引
        preorder_root = preorder_left
        # 根据根节点的值，给出其在中序遍历中的位置
        inorder_root = index[preorder[preorder_root]]
        
        # 先把根节点建立出来
        root = TreeNode(preorder[preorder_root])
        # 中序遍历列表中，根节点左边的元素个数即为左子树节点个数
        size_left_subtree = inorder_root - inorder_left
        
        # 递归地构造左子树，并连接到根节点
        # 先序遍历中「从 左边界+1 开始的 size_left_subtree」个元素（左边界是根节点）就对应了中序遍历中「从 左边界 开始到 根节点定位-1」的元素（根节点左边的是左子树的元素））
        root.left = myBuildTree(preorder_left + 1, preorder_left + size_left_subtree, inorder_left, inorder_root - 1)
        # 递归地构造右子树，并连接到根节点
        # 先序遍历中「从 左边界+1+左子树节点数目 开始到 右边界」的元素就对应了中序遍历中「从 根节点定位+1 到 右边界」的元素
        root.right = myBuildTree(preorder_left + size_left_subtree + 1, preorder_right, inorder_root + 1, inorder_right)
        return root
    
    n = len(preorder)
    # 构造哈希映射，帮助我们快速定位根节点
    index = {element: i for i, element in enumerate(inorder)}  # key是节点的值，value是值对应的中序遍历列表索引
    return myBuildTree(0, n - 1, 0, n - 1)

#### 测试一

In [3]:
preorder = [3,9,20,15,7]
inorder = [9,3,15,20,7]
root = buildTree(preorder, inorder)
print(root.val, root.left.val, root.right.val, root.right.left.val, root.right.right.val)

3 9 20 15 7


# 方法二：迭代

> 对于前序遍历中的任意两个连续节点a和b，根据前序遍历的流程，a和b有两种关系：  
1. b是a的左子节点。
2. a没有左子节点，b是a或a的某个祖先节点的右子节点。这是因为，若a没有左子节点，那么前序遍历顺序下，遍历完a后遍历的下一个节点即是a的右子节点，若a没有右子节点，则向上回溯到一个有右子节点的祖先节点，b是该节点的右子节点。

> 举例说明如下：

>       3  
>      / \  
>      9  20  
>     /  /  \  
>     8  15   7  
>     / \  
>     5  10  
>     /  
>     4  

> 其前序遍历与中序遍历分别为：  
preorder = [3, 9, 8, 5, 4, 10, 20, 15, 7]  
inorder = [4, 5, 8, 10, 9, 3, 15, 20, 7]

> 我们用一个栈 `stack` 来维护「当前节点的所有还没有考虑过右儿子的祖先节点」，栈顶就是当前节点。也就是说，只有在栈中的节点才可能连接一个新的右儿子。同时，我们用一个指针 `index` 指向中序遍历的某个位置，初始值为 0。`index` 对应的节点是「当前节点不断往左走达到的最终节点」，这也是符合中序遍历的，它的作用在下面的过程中会有所体现。  

>1. 将根节点 3 入栈，再初始化 `index` 所指向的节点为 4，随后对于前序遍历中的每个节点，我们依次判断它是栈顶节点的左儿子，还是栈中某个节点的右儿子。我们遍历 9。9 一定是栈顶节点 3 的左儿子。我们使用反证法，假设 9 是 3 的右儿子，那么 3 没有左儿子，`index` 应该恰好指向 3，但实际上为 4，因此产生了矛盾。所以我们将 9 作为 3 的左儿子，并将 9 入栈。  
stack = [3, 9]  
index -> inorder[0] = 4  
2. 我们遍历 8，5 和 4。同理可得它们都是上一个节点（栈顶节点）的左儿子，所以它们会依次入栈。  
stack = [3, 9, 8, 5, 4]  
index -> inorder[0] = 4  
3. 我们遍历 10，这时情况就不一样了。我们发现 `index` 恰好指向当前的栈顶节点 4，也就是说 4 没有左儿子，那么 10 必须为栈中某个节点的右儿子。那么如何找到这个节点呢？栈中的节点的顺序和它们在前序遍历中出现的顺序是一致的，而且每一个节点的右儿子都还没有被遍历过，那么这些节点的顺序和它们在中序遍历中出现的顺序一定是相反的。（这是因为栈中的任意两个相邻的节点，前者都是后者的某个祖先。并且我们知道，栈中的任意一个节点的右儿子还没有被遍历过，说明后者一定是前者左儿子的子树中的节点，那么后者就先于前者出现在中序遍历中）  
因此我们可以把 `index` 不断向右移动，并与栈顶节点进行比较。如果 index 对应的元素恰好等于栈顶节点，那么说明我们在中序遍历中找到了栈顶节点，所以将 `index` 增加 1 并弹出栈顶节点，直到 `index` 对应的元素不等于栈顶节点。按照这样的过程，我们弹出的最后一个节点 `x` 就是 10 的双亲节点，这是因为 10 出现在了 `x` 与 `x` 在栈中的下一个节点的中序遍历之间，因此 10 就是 `x` 的右儿子。  
回到我们的例子，我们会依次从栈顶弹出 4，5 和 8，并且将 `index` 向右移动了三次。我们将 10 作为最后弹出的节点 8 的右儿子，并将 10 入栈。  
stack = [3, 9, 10]  
index -> inorder[3] = 10  
4. 我们遍历 20。同理，`index` 恰好指向当前栈顶节点 10，那么我们会依次从栈顶弹出 10，9 和 3，并且将 `index` 向右移动了三次。我们将 20 作为最后弹出的节点 3 的右儿子，并将 20 入栈。  
stack = [20]  
index -> inorder[6] = 15  
5. 我们遍历 15，将 15 作为栈顶节点 20 的左儿子，并将 15 入栈。  
stack = [20, 15]  
index -> inorder[6] = 15  
6. 我们遍历 7。`index` 恰好指向当前栈顶节点 15，那么我们会依次从栈顶弹出 15 和 20，并且将 `index` 向右移动了两次。我们将 7 作为最后弹出的节点 20 的右儿子，并将 7 入栈。  
stack = [7]  
index -> inorder[8] = 7  
7. 此时遍历结束，我们就构造出了正确的二叉树。

## 复杂度

- 时间复杂度: $O(n)$ ，其中 $n$ 是树中的节点个数。

> 创建了 $n$ 个节点。

- 空间复杂度: $O(n)$ ，其中 $n$ 是树中的节点个数。

> 除去返回的答案需要的 $O(n)$ 空间之外，我们还需要使用 $O(h)$ （其中 $h$ 是树的高度）的空间存储栈， $h<n$ ，所以总空间复杂度为 $O(n)$ 。

## 代码

In [4]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [5]:
def buildTree(preorder, inorder):
    if not preorder:
        return None

    root = TreeNode(preorder[0])  # 构造根节点
    stack = [root]  # 根据前序遍历创建一个栈
    inorderIndex = 0  # 使用指针遍历中序遍历列表
    for i in range(1, len(preorder)):  # 从根节点的下一节点开始遍历前序遍历列表
        preorderVal = preorder[i]
        node = stack[-1]  # 当前节点为栈顶元素
        if node.val != inorder[inorderIndex]:  # 如果栈顶元素的值不等于指针指向的值
            node.left = TreeNode(preorderVal)  # 说明下一节点是当前节点的左子节点
            stack.append(node.left)  # 将下一节点入栈
        else:  # 若栈顶元素的值等于指针指向的值，说明下一节点是当前节点或其祖先节点的右子节点
            # 从栈中弹出元素，每弹出一个元素，指针向右边移动一位，直到弹出的元素值不等于指针指向的元素值
            while stack and stack[-1].val == inorder[inorderIndex]:
                node = stack.pop()
                inorderIndex += 1
            # 指针指向的元素即为最后弹出的元素的右子节点
            node.right = TreeNode(preorderVal)
            stack.append(node.right)  # 将该节点入栈

    return root

#### 测试一

In [6]:
preorder = [3,9,20,15,7]
inorder = [9,3,15,20,7]
root = buildTree(preorder, inorder)
print(root.val, root.left.val, root.right.val, root.right.left.val, root.right.right.val)

3 9 20 15 7
